In [1]:
import polars as pl
import pyodbc
from datetime import date

import time
start_time = time.time()

In [2]:
from IPython import get_ipython
get_ipython().run_line_magic("run", "functions.ipynb")

#### Definición de Rutas

In [3]:
cierre = "202503"
cierre_l = "Mar25"

# Carpetas
path = "E:/Users/jhernandezr/DAR/garantias/reporte/fotos/"
wd_data_raw = path + "data/raw/"
wd_data_external = path + "data/external/"

wd_data_processed_dwh = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/DWH/csv/"

# Archivos
## Inputs
fl_desembolsos_p1 = wd_data_raw + f'Desembolsos_P1_{cierre}'
fl_desembolsos_p1_bmxt = wd_data_raw + f'Desembolsos_P1_80686_{cierre}'
fl_desembolsos_p1_fianzas = wd_data_raw + f'Desembolsos_P1_BMXT_{cierre}'
fl_desembolsos_p2 = wd_data_raw + f'Desembolsos_P2_{cierre}'
fl_desembolsos_p2_bmxt = wd_data_raw + f'Desembolsos_P2_80686_{cierre}'
fl_desembolsos_p2_fianzas = wd_data_raw + f'Desembolsos_P2_BMXT_{cierre}'
fl_catalogos = wd_data_external + f'Catálogos_{cierre_l}'
fl_recuperaciones = wd_data_raw + f'Recuperaciones_{cierre}.csv'
fl_recuperaciones_bmxt = wd_data_raw + f'Recuperaciones_BMXT_{cierre}.mdb'

# Outputs
fl_pagadas_global_vf = wd_data_processed_dwh + f"Pagadas_Global_VF_{cierre}.csv"
fl_valida_pagos = wd_data_processed_dwh + f'Valida_Pagos.csv'
fl_recuperadas_global_vf = wd_data_processed_dwh + f'Recuperadas_Global_VF_{cierre}.csv'
fl_valida_dwh_dac = wd_data_processed_dwh + f'Recuperadas_Valida_DWHvsDAC.csv'
fl_valida_td = wd_data_processed_dwh + f'Recuperadas_Valida_TD.csv'
fl_recupera_con_pagos_flujos = wd_data_processed_dwh + f'Recupera_con_Pagos_Flujos_{cierre}.csv'
fl_valida_base_pagos = wd_data_processed_dwh + f'Valida_Base_Pagos.csv'
fl_valida_base_pagos_mn = wd_data_processed_dwh + f'Valida_Base_Pagos_Mn.csv'
fl_valida_base_recup = wd_data_processed_dwh + f'Valida_Base_Recup.csv'
fl_valida_base_recup_mn = wd_data_processed_dwh + f'Valida_Base_Recup_Mn.csv'




### Catálogos

In [4]:
conn_str = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + fl_catalogos + '.accdb'
conn = pyodbc.connect(conn_str)

# Solo Recuperaciones
estatus = pl.read_database(query='SELECT * FROM [ESTATUS]', connection=conn)

# Recuperaciones, Pagos
tipo_cambio = pl.read_database(query='SELECT * FROM [TIPO CAMBIO]', connection=conn)
programa = pl.read_database(query='SELECT * FROM [PROGRAMA]', connection=conn, schema_overrides={'PROGRAMA_ID': pl.Int128})
agrupamiento = pl.read_database(query='SELECT * FROM [AGRUPAMIENTO]', connection=conn)
udis = pl.read_database(query='SELECT * FROM [UDIS]', connection=conn, schema_overrides={'Fecha_Paridad': pl.Datetime})
#udis = udis.with_columns(pl.col("Fecha_Paridad").cast(pl.Date))
tipo_credito = pl.read_database(query='SELECT * FROM [TIPO_CREDITO]', connection=conn, schema_overrides={'Tipo_Credito_ID': pl.Int32})
tipo_garantia = pl.read_database(query='SELECT * FROM [TIPO_GARANTIA]', connection=conn, schema_overrides={'Tipo_garantia_ID': pl.Int32})
sfc = pl.read_database(query='SELECT * FROM [SIN FONDOS CONTRAGARANTIA]', connection=conn)

conn.close()

tipo_cambio.columns = ["Anio", "Mes", "TC"]
cierre = date(2025,3,31)
tdc = tipo_cambio.filter((pl.col("Anio")==cierre.year) & (pl.col("Mes")==cierre.month)).select("TC").item()
tdc

20.4604

### Querie Pagos

In [5]:
def main_querie_pagadas():
    pagos_f1 = import_f1()
    pagos_f2 = import_f2()

    # Crea Concatenados y TPRO_CLAVE
    pagos_f1 = genera_concatenado(pagos_f1)
    pagos_f1 = genera_tpro_clave(pagos_f1)
    pagos_f2 = genera_concatenado(pagos_f2)

    pagadas_global_inter = genera_pagadas_global_inter(pagos_f1, pagos_f2)
    pagadas_global_vf = genera_pagadas_global_vf(pagadas_global_inter)
    pagadas_global_vf = complementa_pagadas_global_vf(pagadas_global_vf)

    valida_pagos = genera_valida_pagos(pagos_f2)
    # pagadas_global_vf_bancomext = genera_pagadas_global_bancomext(pagadas_global_vf)
    # pagadas_global_vf_sin_bancomext = genera_pagadas_global_sin_bancomext(pagadas_global_vf)

    valida_base_pagos = genera_valida_base_pagos(pagadas_global_vf)
    valida_base_pagos_mn = genera_valida_base_pagos_mn(pagadas_global_vf)

    pagadas_global_vf.write_csv(fl_pagadas_global_vf, datetime_format="%Y-%m-%d", float_precision=10)
    valida_pagos.write_csv(fl_valida_pagos)
    #pagadas_global_vf_bancomext.write_csv('./../data/processed/DWH/pagadas_global_vf_bancomext.csv')
    #pagadas_global_vf_sin_bancomext.write_csv('./../data/processed/DWH/pagadas_global_vf_sin_bancomext.csv')
    
    valida_base_pagos.write_csv(fl_valida_base_pagos, float_precision = 10)
    valida_base_pagos_mn.write_csv(fl_valida_base_pagos_mn, float_precision = 10)

    return pagadas_global_vf

pagadas_global_vf = main_querie_pagadas()

### Querie Recuperaciones

#### Ejecución

In [6]:
def main_querie_recuperaciones():
    dwh_recuperaciones = importa_recuperaciones()
    recuperadas_global_inter = genera_recuperadas_global_inter(dwh_recuperaciones)
    recuperadas_global_vf = genera_recuperadas_global_vf(recuperadas_global_inter)

    #recuperadas_global_vf_bancomext = genera_recuperadas_global_bancomext(recuperadas_global_vf)
    #recuperadas_global_vf_sin_bancomext = genera_recuperadas_global_sin_bancomext(recuperadas_global_vf)

    valida_dwh_dac = genera_valida_dwh_dac(dwh_recuperaciones)
    valida_td = genera_valida_td(dwh_recuperaciones)

    recuperadas_global_vf.write_csv(fl_recuperadas_global_vf, datetime_format="%Y-%m-%d", float_precision=6)
    valida_dwh_dac.write_csv(fl_valida_dwh_dac, float_precision=6, float_scientific=False)
    valida_td.write_csv(fl_valida_td,float_precision=6, float_scientific=False)
    #recuperadas_global_vf_bancomext.write_csv('./../data/processed/DWH/recuperadas_global_vf_bancomext.csv')
    #recuperadas_global_vf_sin_bancomext.write_csv('./../data/processed/DWH/recuperadas_global_vf_sin_bancomext.csv')
    
    return recuperadas_global_vf
#recuperadas_global_vf = main_querie_recuperaciones()

### Querie Union Flujos

#### Ejecución

In [7]:
def genera_concatenado(df, intermediario="INTERMEDIARIO_ID", no_credito="NUMERO_CREDITO"):
    result = (df.with_columns(
                (pl.col(intermediario) + pl.col(no_credito))
                .alias("Concatenado"))
            )
    
    return result

In [8]:
def main_querie_union_flujos(pagadas_global_vf, recuperadas_global_vf):
    pagos_agrup = genera_pagos_agrup(pagadas_global_vf)
    pagos_agrup = genera_concatenado(pagos_agrup, intermediario="Intermediario_Id", no_credito="Numero_Credito")

    recuperadas_global_vf = genera_concatenado(recuperadas_global_vf)
    #recuperaciones_agrup = genera_recuperaciones_agrup(recuperadas_global_vf)

    # Falta depurar columnas de B
    #uf_pagos_recuperaciones = pagos_agrup.join(recuperadas_global_vf, on="Concatenado", how='left')

    # Falta depurar columnas de B
    uf_recuperaciones_pagos = recuperadas_global_vf.join(pagos_agrup, on="Concatenado", how='left')

    #t_roberto = pagos_agrup.join(recuperaciones_agrup, on="Concatenado", how='left')

    recupera_con_pagos_flujos = genera_recupera_con_pagos_flujos(uf_recuperaciones_pagos)
    #recupera_con_pagos_flujos_bmxt = genera_recupera_con_pagos_flujos_bancomext(recupera_con_pagos_flujos)
    #recupera_con_pagos_flujos_sin_bmxt = genera_recupera_con_pagos_flujos_sin_bancomext(recupera_con_pagos_flujos)

    valida_base_pagos_recup = genera_valida_base_pagos_recup(recuperadas_global_vf)
    valida_base_pagos_recup_mn = genera_valida_base_pagos_recup_mn(recuperadas_global_vf)
    
    recupera_con_pagos_flujos.write_csv(fl_recupera_con_pagos_flujos, datetime_format="%Y-%m-%d", float_precision=6)
    # recupera_con_pagos_flujos_bmxt.write_csv('./../data/processed/DWH/recupera_con_pagos_flujos_bmxt.csv')
    # recupera_con_pagos_flujos_sin_bmxt.write_csv('./../data/processed/DWH/recupera_con_pagos_flujos_sin_bmxt.csv')
    valida_base_pagos_recup.write_csv(fl_valida_base_recup)
    valida_base_pagos_recup_mn.write_csv(fl_valida_base_recup_mn)

    return 0
#main_querie_union_flujos(pagadas_global_vf, recuperadas_global_vf)


In [9]:
#main_querie_union_flujos(pagadas_global_vf, recuperadas_global_vf)

### Ejecución Principal

In [10]:
if __name__ ==  "__main__":
    pagadas_global_vf = main_querie_pagadas()
    recuperadas_global_vf = main_querie_recuperaciones()
    main_querie_union_flujos(pagadas_global_vf, recuperadas_global_vf)

In [11]:
end_time = time.time()
end_time - start_time

39.50812077522278